In [1]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
%matplotlib inline
plt.rcParams['figure.dpi']= 150

First, we define parameters:

In [2]:
# Transport parameters
s_perm = 0.1
u_perm = 0.1
w_perm_D = 0.1
w_perm_C = 0.1
S_input = 0.1
U_input = 2
C_osm = 2

# discretization
N = 10
dx = 1/N

Now, we define the difference quotient operator and the average operator:

In [3]:
def diff_x(y): 
    n = len(y)-1
    diff = np.zeros(n)
    for l in range(n):
        diff[l] = (y[l+1]-y[l])/dx
    
    return diff


def avg(y):
    n = len(y)-1
    average = np.zeros(n)
    for l in range(n):
        average[l] = (y[l+1]+y[l])/2
    
    return average

Next, we define $u_0$, $s_0$:

In [4]:
def su_0(q_D,s,q_C,u,q_A,q_out,U_out):
    D_avg = avg(q_D)
    s_avg = avg(s)
    u_avg = avg(u)
    C_avg = avg(q_C)
    A_avg = avg(q_A)
    q_out_avg = avg(q_out)
    U_out_avg = avg(U_out)

    u_0 = np.zeros(N)
    s_0 = np.zeros(N)
    # Q = q_D[:-1]-q_D[-1]+q_C[:-1]-q_C[-1]
    # Q = q_D[:-1]-q_A[:-1]+q_C[:-1]-q_out[:-1]
    Q = D_avg+A_avg+C_avg-q_out_avg
    # Q = D_avg - q_D[-1] + C_avg - q_C[-1]
    # s_0[:] = (S_input + s[:-1]*q_A[:-1])/Q
    if s_perm != 0:
        s_0[:] = (S_input + s_avg*A_avg)/Q
        # s_0[:] = (S_input - s_avg*q_D[-1])/Q
    # u_0[:] = (u[:-1]*q_C[:-1]-u[-1]*q_C[-1])/Q
    # u_0[:] = (u[:-1]*q_C[:-1]-U_out[:-1])/Q
    if u_perm != 0:
        u_0[:] = (u_avg*C_avg-U_out_avg)/Q
        # u_0[:] = (u_avg*C_avg-u[-1]*q_C[-1])/Q
    s_0 = np.abs(s_0)
    u_0 = np.abs(u_0)

    return s_0,u_0

In [5]:
def res(y):
    q_D = y[:(N+1)].copy()
    # q_D = np.abs(y[:(N+1)])
    s = y[(N+1):2*(N+1)].copy()
    # s = np.abs(y[(N+1):2*(N+1)])
    q_C = y[2*(N+1):3*(N+1)].copy()
    # q_C = np.abs(y[2*(N+1):3*(N+1)])
    u = y[3*(N+1):4*(N+1)].copy()
    # u = np.abs(y[3*(N+1):4*(N+1)])
    q_A = y[4*(N+1):5*(N+1)].copy()
    # q_A = -np.abs(y[4*(N+1):5*(N+1)])
    q_out = y[5*(N+1):6*(N+1)].copy()
    U_out = y[6*(N+1):7*(N+1)].copy()
    s_0,u_0 = su_0(q_D,s,q_C,u,q_A,q_out,U_out)

    residue = np.zeros(7*(N+1))
    res_D = residue[:(N+1)]
    res_s = residue[(N+1):2*(N+1)]
    res_C = residue[2*(N+1):3*(N+1)]
    res_u = residue[3*(N+1):4*(N+1)]
    res_A = residue[4*(N+1):5*(N+1)]
    res_q_out = residue[5*(N+1):6*(N+1)]
    res_U_out = residue[6*(N+1):7*(N+1)]

    w_D = w_perm_D*(2*s_0 + u_0 - (2*S_input/avg(q_D)))
    w_C = w_perm_C*(2*s_0 + u_0 - avg(u))
    j_s = s_perm*(avg(s) - s_0)
    j_u = u_perm*(avg(u) - u_0)

    res_D[0] = q_D[0] - 2*S_input/C_osm
    # res_D[1:] = diff_x(q_D) + w_D
    res_D[1:] = w_D
    # res_s[0] = s[-1]+S_input/q_A[-1]
    res_s[0] = s[-1] - S_input/q_D[-1]
    # res_s[1:] = diff_x(s) - (s_perm/q_D[-1])*(s[:-1] - s_0)
    res_s[1:] = diff_x(s) + j_s/avg(q_A)
    res_C[0] = q_C[0] - U_input/C_osm
    res_C[1:] = diff_x(q_C) + w_C
    res_u[0] = u[0] - C_osm
    res_u[1:] = diff_x(u) - (1/avg(q_C))*(
        avg(u)*w_C - j_u
    )

    res_A[0] = q_A[-1] + q_D[-1]
    res_A[1:] = diff_x(q_A)
    res_q_out[0] = q_out[-1] - q_C[-1]
    res_q_out[1:] = diff_x(q_out)
    res_U_out[0] = U_out[-1] - u[-1]*q_C[-1]
    res_U_out[1:] = diff_x(U_out)

    return residue

Finally, we put everything together:

In [6]:
# Transport parameters
s_perm = 0.1
u_perm = 0.1

w_perm_D = 0.1
w_perm_C = 0.1

S_input = 0.1
U_input = 2
C_osm = 2


# discretization
N = 10
dx = 1/N

#Initial guess
y_init = np.zeros(7*(N+1))
q_D_init = y_init[:(N+1)]
s_init = y_init[(N+1):2*(N+1)]
q_C_init = y_init[2*(N+1):3*(N+1)]
u_init = y_init[3*(N+1):4*(N+1)]
q_A_init = y_init[4*(N+1):5*(N+1)]
q_out_init = y_init[5*(N+1):6*(N+1)]
U_out_init = y_init[6*(N+1):7*(N+1)]
for n in range(N+1):
    q_D_init[n] = (1- n/(1.2*N))*2*S_input/C_osm
    # q_D_init[n] = 2*S_input/C_osm
    s_init[n] = 0.1*n + C_osm/2
    # s_init[n] = C_osm/2
    q_C_init[n] = (1- n/(1.2*N))*U_input/C_osm
    # q_C_init[n] = U_input/C_osm
    u_init[n] = 0.2*n + C_osm
    # u_init[n] = C_osm
q_A_init[:] = -q_D_init[-1].copy()
q_out_init[:] = q_C_init[-1].copy()
U_out_init[:] = u_init[-1]*q_C_init[-1]

y = fsolve(res,y_init)
q_D = y[:(N+1)].copy()
s = y[(N+1):2*(N+1)].copy()
q_C = y[2*(N+1):3*(N+1)].copy()
u = y[3*(N+1):4*(N+1)].copy()
q_A = y[4*(N+1):5*(N+1)].copy()
q_out = y[5*(N+1):6*(N+1)].copy()
U_out = y[6*(N+1):7*(N+1)].copy()
s_0,u_0 = su_0(q_D,s,q_C,u,q_A,q_out,U_out)
osm = 2*s_0 + u_0

c:\Users\馬予棟\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


Let's plot the solution.